In [36]:
try:
  import overpass
except:
  !pip install overpass
  import overpass
from shapely.geometry import LineString
import pandas as pd
import numpy as np
import json
try:
  from SPARQLWrapper import SPARQLWrapper, JSON
except:
  !pip install SPARQLWrapper
  from SPARQLWrapper import SPARQLWrapper, JSON

In [37]:
QosmCemeteries = """
area[wikidata={0}]->.searchArea;
(
  nwr[landuse=cemetery](area.searchArea);
  nwr[amenity=graveyard](area.searchArea);
)->.cemeteries;
.cemeteries out center;"""
QosmGraves = """
area[wikidata={0}]->.searchArea;
(
  nwr[historic=tomb](area.searchArea);
  nwr[cemetery=grave](area.searchArea);
);
"""
QwdBuriedPeople = """SELECT
  ?item
  ?itemLabel
  ?itemDescription
  ?burialPlace
  ?burialPlaceLabel
  ?location
  ( COUNT( ?sitelink ) AS ?sitelink_count )
WHERE
{{
  ?item wdt:P31 wd:Q5. #instance of human
  ?item wdt:P119 ?burialPlace. #place of burial
  ?burialPlace wdt:P131+ wd:{0}. #place of burial in config city
  OPTIONAL{{?item p:P119 [ ps:P119 ?placeburial; pq:P625 ?location ].}} #return the coordinate location qualifier of place of burial if it exists
  ?sitelink schema:about ?item. #get site links
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }} #default label thing
  Optional{{?item schema:description"en".}} #get English description
}}
GROUP BY ?item ?itemLabel ?location ?burialPlace ?burialPlaceLabel ?itemDescription ?burialPlaceCoords #group items, if other field are to be shown they need to be added to this
ORDER BY DESC( COUNT( ?sitelink ) ) #sort by number of sitelinks for each item
"""

QwdCemeteries = """SELECT DISTINCT ?item ?itemLabel ?coordLocation WHERE {{
  ?item wdt:P131+ wd:{0}. #located in config city or area in config city
  ?item (wdt:P31|wdt:P31/wdt:P279|wdt:P31/wdt:P279/wdt:P279) wd:Q39614. #instance of cemetery, only goes 2 layers into subclasses to limit timeout issues
  OPTIONAL{{?item wdt:P625 ?coordLocation.}}
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}"""


In [38]:
areaQID = "Q60" # @param {type:"string"}

In [39]:
overpassAPI = overpass.API()
def extractQID(s):
	start = s.rfind('/')+1
	end = len(s)
	s = s[start:end]
	return s
sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql", agent = "https://github.com/hdevine825/wd-osm-burials")
sparql.setReturnFormat(JSON)
nan=np.nan

In [40]:
sparql.setQuery(QwdBuriedPeople.format(areaQID))
results = sparql.query().convert()
wddf = pd.json_normalize(results['results']['bindings'])
wddf['personQID'] = wddf.apply(lambda row: extractQID(row['item.value']),axis=1)
wddf['burialPlaceQID'] = wddf.apply(lambda row: extractQID(row['burialPlace.value']),axis=1)
print("Wikidata returned {0} people buried in {1} places".format(wddf.shape[0],wddf['burialPlace.value'].nunique()))

Wikidata returned 2507 people buried in 81 places


In [41]:
sparql.setQuery(QwdCemeteries.format(areaQID))
results = sparql.query().convert()
wdcemdf = pd.json_normalize(results['results']['bindings'])
wdcemdf['cemeteryQID'] = wdcemdf.apply(lambda row: extractQID(row['item.value']),axis=1)
print("Wikidata returned {0} cemeteries".format(wdcemdf.shape[0]))

Wikidata returned 77 cemeteries


In [42]:
osmcem = overpassAPI.get(QosmCemeteries.format(areaQID),verbosity="geom center",responseformat="json")
osmcemdf = pd.json_normalize(osmcem['elements'])
print("Overpass returned {0} cemeteries".format(osmcemdf.shape[0]))

Overpass returned 109 cemeteries


In [43]:
osmGraves = overpassAPI.get(QosmGraves.format(areaQID),verbosity="geom center",responseformat="json")

In [44]:
osmGravesdf = pd.json_normalize(osmGraves['elements'])
print("Overpass returned {0} graves".format(osmGravesdf.shape[0]))

Overpass returned 802 graves


In [45]:
mergedCemdf = pd.merge(left=wdcemdf,right=osmcemdf,left_on='cemeteryQID',right_on='tags.wikidata')
for index, row in mergedCemdf.iterrows():
	print("{0}: WD: {1} OSM: {2}, {3}".format(row['itemLabel.value'],row['coordLocation.value'],row['center.lon'],row['center.lat']))
	burialsdf = wddf.query("burialPlaceQID=='{0}'".format(row["cemeteryQID"]))
	#for pindex, prow in burialsdf.iterrows():
		#print(" "+prow["itemLabel.value"])
	#Incorporate measuring distance between points here

Saint Michael's Cemetery: WD: Point(-73.8994 40.7647) OSM: -73.8989193, 40.7637089
Holy Cross Cemetery: WD: Point(-73.938569444 40.646488888) OSM: -73.938083, 40.647115
Silver Mount Cemetery: WD: Point(-74.0956 40.6219) OSM: -74.0952168, 40.6219986
The Evergreens Cemetery: WD: Point(-73.901194 40.683889) OSM: -73.8989755, 40.6865328
First Shearith Israel Graveyard: WD: Point(-73.998333 40.7125) OSM: -73.9983586, 40.7127712
Trinity Church Cemetery: WD: Point(-74.011667 40.708333) OSM: -74.0120391, 40.7081698
Baron Hirsch Cemetery: WD: Point(-74.1549 40.6221) OSM: -74.1559323, 40.6220845
Cypress Hills National Cemetery: WD: Point(-73.8753 40.689) OSM: -73.8821044, 40.6876036
Flushing Cemetery: WD: Point(-73.799444444 40.751666666) OSM: -73.799431, 40.7516635
Green-Wood Cemetery: WD: Point(-73.99024 40.65241) OSM: -73.9923642, 40.651906
New York Marble Cemetery: WD: Point(-73.990972 40.725625) OSM: -73.9909804, 40.7256318
Mount Saint Mary Cemetery: WD: Point(-73.8017 40.7406) OSM: -73.800

In [46]:
osmGravesdf['tags.subject:wikidata'].fillna(osmGravesdf['tags.buried:wikidata'],axis=0,inplace=True)
mergedBurialdf = pd.merge(how='outer',left=wddf,right=osmGravesdf,left_on='personQID',right_on='tags.subject:wikidata')

In [47]:
fullCoords = mergedBurialdf.query("`tags.subject:wikidata`==`tags.subject:wikidata` & `location.value`==`location.value`")
osmNoWDcoords = mergedBurialdf.query("`tags.subject:wikidata`==`tags.subject:wikidata` & `location.value`!=`location.value` & `item.type`==`item.type`")
osmNoWD = mergedBurialdf.query("`tags.subject:wikidata`==`tags.subject:wikidata` & `item.type`!=`item.type`")
osmNoQID = mergedBurialdf.query("`tags.subject:wikidata`!=`tags.subject:wikidata` & id==id")
print("People with WD coordinates and burial on OSM: {0}".format(fullCoords.shape[0]))
print("Burials on OSM without coordinates in Wikidata: {0}".format(osmNoWDcoords.shape[0]))
print("Burials with WD tags on OSM not in WD query: {0}".format(osmNoWD.shape[0]))
print("Burials on OSM without WD tags: {0}".format(osmNoQID.shape[0]))
print("   With name=*: {0}".format(osmNoQID.query("`tags.name`==`tags.name`").shape[0]))

People with WD coordinates and burial on OSM: 11
Burials on OSM without coordinates in Wikidata: 2
Burials with WD tags on OSM not in WD query: 1
Burials on OSM without WD tags: 789
   With name=*: 20
